# FireProx Phase 1 Demo - Synchronous API

This notebook demonstrates all Phase 1 features of the FireProx synchronous API.

**Prerequisites**: Firestore emulator must be running on port 8080.

## 1. Setup and Initialization

In [ ]:
from google.cloud import firestore
from fire_prox import FireProx, State

# Initialize native Firestore client (assumes emulator is running)
client = firestore.Client(project='demo-project')
db = FireProx(client)
print("FireProx initialized successfully!")

## 2. Creating a New Document (DETACHED State)

In [ ]:
# Get a collection reference
users = db.collection('users')

# Create a new document (not yet in Firestore)
user = users.new()
print(f"State: {user.state}")
print(f"Is detached: {user.is_detached()}")
print(f"Is dirty: {user.is_dirty()}")

## 3. Setting Attributes on a DETACHED Document

In [ ]:
# Set attributes using dot notation
user.name = 'Ada Lovelace'
user.year = 1815
user.occupation = 'Mathematician'

print(f"Name: {user.to_dict()['name']}")
print(f"Data: {user.to_dict()}")
print(f"Still detached: {user.is_detached()}")

## 4. Saving with Custom ID (DETACHED → LOADED)

In [ ]:
# Save with a custom document ID
user.save(doc_id='alovelace')

print(f"State after save: {user.state}")
print(f"Is loaded: {user.is_loaded()}")
print(f"Is dirty: {user.is_dirty()}")
print(f"Document ID: {user.id}")
print(f"Document path: {user.path}")

## 5. Getting a Document by Path (ATTACHED State)

In [ ]:
# Get a document reference (doesn't fetch data yet)
user2 = db.doc('users/alovelace')

print(f"State: {user2.state}")
print(f"Is attached: {user2.is_attached()}")
print(f"Document ID: {user2.id}")
print(f"Document path: {user2.path}")
print("Data not fetched yet!")

## 6. Lazy Loading (ATTACHED → LOADED)

In [ ]:
# Accessing an attribute automatically fetches data (lazy loading)
name = user2.name

print(f"Name: {name}")
print(f"State after access: {user2.state}")
print(f"Is loaded: {user2.is_loaded()}")
print(f"Full data: {user2.to_dict()}")

## 7. Explicit Fetch (Alternative to Lazy Loading)

In [ ]:
# Create another reference
user3 = db.doc('users/alovelace')
print(f"Before fetch - State: {user3.state}")

# Explicitly fetch data
user3.fetch()
print(f"After fetch - State: {user3.state}")
print(f"Data: {user3.to_dict()}")

## 8. Modifying a LOADED Document

In [ ]:
# Modify attributes
user3.year = 1816
user3.contributions = ['Analytical Engine', 'First Algorithm']

print(f"Is dirty: {user3.is_dirty()}")
print(f"Modified year: {user3.year}")
print(f"Contributions: {user3.contributions}")

## 9. Saving Updates

In [ ]:
# Save the modifications
user3.save()

print(f"Is dirty after save: {user3.is_dirty()}")
print(f"State: {user3.state}")
print("Changes saved to Firestore!")

## 10. Refreshing Data with force=True

In [ ]:
# Fetch fresh data from Firestore (force refresh)
user3.fetch(force=True)

print(f"Refreshed data: {user3.to_dict()}")
print(f"Year after refresh: {user3.year}")
print(f"Contributions: {user3.contributions}")

## 11. Deleting Attributes

In [ ]:
# Delete an attribute
del user3.contributions

print(f"Is dirty: {user3.is_dirty()}")
print(f"Data after delete: {user3.to_dict()}")
print("Attribute 'contributions' removed locally")

In [ ]:
# Save to persist the deletion
user3.save()
user3.fetch(force=True)

print(f"Data after save: {user3.to_dict()}")
print("'contributions' field removed from Firestore")

## 12. Creating Document with Auto-Generated ID

In [ ]:
# Create a new document without specifying ID
user4 = users.new()
user4.name = 'Grace Hopper'
user4.year = 1906

# Save without doc_id - Firestore generates ID
user4.save()

print(f"Auto-generated ID: {user4.id}")
print(f"Path: {user4.path}")
print(f"Data: {user4.to_dict()}")

## 13. Collection Properties

In [ ]:
# Inspect collection properties
print(f"Collection ID: {users.id}")
print(f"Collection path: {users.path}")
print(f"String repr: {str(users)}")
print(f"Repr: {repr(users)}")

## 14. Deleting a Document (LOADED → DELETED)

In [ ]:
# Delete a document from Firestore
user4.delete()

print(f"State after delete: {user4.state}")
print(f"Is deleted: {user4.is_deleted()}")
print(f"ID still accessible: {user4.id}")
print(f"Path still accessible: {user4.path}")

## 15. Error Handling - Invalid Operations on DELETED

In [ ]:
# Attempting operations on DELETED document raises errors
try:
    user4.save()
except RuntimeError as e:
    print(f"Save error: {e}")

try:
    user4.fetch()
except RuntimeError as e:
    print(f"Fetch error: {e}")

## 16. Hydration from Native Firestore Snapshot

In [ ]:
# Use native Firestore API to get a snapshot
from fire_prox import FireObject

doc_ref = client.collection('users').document('alovelace')
snapshot = doc_ref.get()

# Hydrate to FireObject
user5 = FireObject.from_snapshot(snapshot)

print(f"State: {user5.state}")
print(f"Is loaded: {user5.is_loaded()}")
print(f"Data: {user5.to_dict()}")
print("Hydrated from native snapshot!")

## 17. Working with Nested Data

In [ ]:
# Create document with nested data structures
user6 = users.new()
user6.name = 'Alan Turing'
user6.address = {
    'city': 'London',
    'country': 'UK'
}
user6.achievements = ['Turing Machine', 'Enigma', 'Turing Test']

user6.save(doc_id='aturing')
print(f"Nested data saved: {user6.to_dict()}")

## 18. Accessing Nested Data

In [ ]:
# Access nested fields
user7 = db.doc('users/aturing')
user7.fetch()

print(f"City: {user7.address['city']}")
print(f"First achievement: {user7.achievements[0]}")
print(f"All achievements: {user7.achievements}")

## Summary

This demo covered all Phase 1 features:

✅ **State Machine**: DETACHED → ATTACHED → LOADED → DELETED  
✅ **Dynamic Attributes**: Set/get/delete using dot notation  
✅ **Lazy Loading**: Automatic fetch on attribute access (sync)  
✅ **Explicit Fetch**: Manual data loading with `fetch()`  
✅ **Save Operations**: Create with custom/auto ID, update existing  
✅ **Delete Operations**: Remove documents from Firestore  
✅ **State Inspection**: `state`, `is_loaded()`, `is_dirty()`, etc.  
✅ **Collection Interface**: `new()`, `doc()`, properties  
✅ **Hydration**: `from_snapshot()` for native query results  
✅ **Nested Data**: Dictionaries and lists as attributes  
✅ **Error Handling**: Clear messages for invalid operations  

**Next**: See Phase 2 features (subcollections, queries, partial updates)